## Дерево решений

Задание
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn (это возможно, так как мы реализуем только малую часть функциональности)
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала так же или качественнее, чем у дерева из sklearn
5. Применить реализованное дерево решений для задачи Titanic на kaggle. Применить для той же задачи дерево решений из sklearn. Применить кросс-валидацию для подбора параметров. Сравнить с результатами предыдущих моделей. Если результат улучшился - сделать сабмит. Написать отчет о результатах.

In [2]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

In [442]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0,
                 criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split  # ограничение минимального числа объектов в листе
        self.max_depth = max_depth  # параметр max_depth ограничивает глубину дерева
        self.sufficient_share = sufficient_share
        self.num_class = -1
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print 'invalid criterion name'
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print 'invalid max_features name'
            raise

    # Набор функция для вычисления меры "нечистоты" (Impurity)
    # или меры неоднородности при разделении в узле дерева на левый и правый узлы
    # В функциях используются следующие входные параметры:
    # l_c == l_class_count - количество образцов первого и второго класса в левой части разделённого массива
    # l_s == l_sizes - размер левой части
    # r_c == r_class_count - количество образцов первого и второго класса в правой части разделённого массива
    # r_s == r_sizes - размер правой части
    # передаются массивы таких значений для всех возможных разбиений по местам изменения целевого признака
    # возвращают значения меры неоднородности для различных возможных разбиений
    
    # мера неоднородности Джини
    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        # Общий размер левой и правой части
        size_all = l_s[0, 0] + r_s[0, 0]
        # Доля образцов, котороые принадлежат классу 1 в левом узле
        p_1_left = l_c[:, 0] / l_s[:, 0]
        # Доля образцов, котороые принадлежат классу 2 в левом узле
        p_2_left = l_c[:, 1] / l_s[:, 0]
        # Мера Джини для левого узла
        gini_left = 1.0 - np.square(p_1_left) - np.square(p_2_left)
        # Аналогично для правого узла
        p_1_right = r_c[:, 0] / r_s[:, 0]
        p_2_right = r_c[:, 1] / r_s[:, 0]
        gini_right = 1.0 - np.square(p_1_right) - np.square(p_2_right)
       
        gini_sum = (l_s[:, 0] / size_all) * gini_left + (r_s[:, 0] / size_all) * gini_right      
        return gini_sum
    
    
    # Информационная энтропия
    def __entropy(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        threshold = 1e-10
        # Общий размер левой и правой части
        size_all = l_s[0, 0] + r_s[0, 0]
        # Доля образцов, котороые принадлежат классу 1 в левом узле
        p_1_left = l_c[:, 0] / l_s[:, 0]
        # Доля образцов, котороые принадлежат классу 2 в левом узле
        p_2_left = l_c[:, 1] / l_s[:, 0]
        # Энтропия для левого узла
        entropy_left = -1.0 * (p_1_left * np.log2(np.clip(p_1_left, threshold, 1.0 - threshold)) +
                              p_2_left * np.log2(np.clip(p_2_left, threshold, 1.0 - threshold)))
        # Аналогично для правого узла
        p_1_right = r_c[:, 0] / r_s[:, 0]
        p_2_right = r_c[:, 1] / r_s[:, 0]
        entropy_right = -1.0 * (p_1_right * np.log2(np.clip(p_1_right, threshold, 1.0 - threshold)) +
                              p_2_right * np.log2(np.clip(p_2_right, threshold, 1.0 - threshold)))
       
        entropy_sum = (l_s[:, 0] / size_all) * entropy_left + (r_s[:, 0] / size_all) * entropy_right      
        return entropy_sum

    
    # Ошибка классификации (misclassification error)
    def __misclass(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        # Общий размер левой и правой части
        size_all = l_s[0, 0] + r_s[0, 0]
        # Неоднородность левой части после разделения
        # (выбираем преобладающий класс в левой части и делим на размер левой части == максимальная
        # вероятность класса в левой части после разделения)
        ME_left = 1.0 - np.max(l_c, axis=1) / l_s[:,0]
        # Неоднородность правой части после разделения
        ME_right = 1.0 - np.max(r_c, axis=1) / r_s[:,0]
        # Общая неоднородность левой и правой частей
        ME_sum = (l_s[:, 0] / size_all) * ME_left + (r_s[:, 0] / size_all) * ME_right   
        return ME_sum

    
    # Дла разделения в узлах могут быть выбираны не все признаки, а только несколько из них
    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return # Ваш код
      
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return # Ваш код

    
    def __get_feature_ids_N(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids
 

    # Выполняет сортировку признака по возрастанию. На вход передаётся один признак x
    def __sort_samples(self, x, y):  
        sorted_idx = x.argsort() 
        return x[sorted_idx], y[sorted_idx]

    
    # метод делит выборку по некоторму пороговому значению
    # выбираются те строки, где значение признака feature_id больше порогового значения (left_mask),
    # и те строки, где значение признака featuer_id меньше порогового значения (rigth_mask)
    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    
    # Метод находит пороги (т.е. значения признака, при которых целевой признак y
    # меняет своё значение) для признака x
    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        # 1. Сортирует x и y в порядке возрастания x
        # 2. Вычисляется общее количество классов в y (в нашем случае 2 класса - 0 и 1)
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = np.unique(y).shape[0]
        
        # Что делает этот блок кода?
        # 1. Выбираем из массива sorted_y значения в середине, кроме первых и последних min_samples_split
        # строк (чтобы соблюсти ограничение на минимальное кол-во точек в листе дерева)
        # 2. В np.where(...) мы находим индексы в массиве y, в которых значение y изменяется
        # (сравнивается массив y и он же, но смещённый на одну позицию. Т.е. каждый элемент y сравнивается
        # со следующим элементом y). Далее индекс приводится к позиции в исходной матрице.
        # В итоге r_border_ids содержит индекс элемента в массиве sorted_y, в котором значение y изменилось
        # по сравнению со значением в предыдущем элементе
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
        
        # Если для данного признака x целевой признак ни разу не изменяется, то такой признак 
        # не имеет смысла рассматривать для построения дерева
        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        # Что делает этот блок кода?
        # 1. eq_el_count - вычисляется, как долго в массиве y сохраняется постоянное значение до очередного изменения
        # 2,3. one_hot_code - строит массив из двух столбцов с чередующимися 0 и 1, смещёнными на одну позицию
        # в первом и втором столбце (своеобразная "змейка")
        # 4. Первый столбец one_hot_code умножается на столбец eq_el_count
        # 5. К первой строке ещё добавляется поправка на количество разных классво в первых 
        # min_samples_split строках sorted_y
        # В итоге в class_increments содержится таблица, в которой показано, сколько раз сохраняется каждое
        # значение в sorted_y перед следующим изменением
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:self.min_samples_split], minlength=class_number)
        
        # Что делает этот блок кода?
        # Вычисляются следующие показатели:
        # l_class_count - количество элементво 1-го и 2-го классов в левой части разделённого массива
        # r_class_count - количество элементов 1-го и 2-го классов в правой части разделённого массива
        # l_sizes = размер левой половины разделённого массива
        # r_sizes = размер правой половины разделённого массива
        # и, т.к. разделение возможно во всех местах изменения целевого признака y, то эти показатели
        # вычисляются для всех мест разделения
        l_class_count = np.cumsum(class_increments, axis=0)        
        r_class_count = np.bincount(y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        # 1. Вычисляется мера неоднородности при разделении класса как сумма неоднородностей
        # в левой и правой половинах
        # 2. Индекс разделения, дающего минимальную неоднородность (лучшее разделение)
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)
    
        # Что делает этот блок кода?
        # 1. Размер левой половины разделённого массива
        # 2. Возвращает значение меры неоднородности, а также среднее значение признака x
        # слева и справа от границы разделения (пороговое значение)
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    
    # Метод, который строит дерево решений?
    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # В начале проверяем, не достигнуто ли условие окончания построения дерева
        # (окончание рекурсивных вызовов)
        
        # 1. Если достигнут один из признаков завершения, то создаём лист
        is_it_leaf = False
        
        # достигнута максимальная глубина дерева
        if (self.max_depth is not None) and (depth >= self.max_depth):
            is_it_leaf = True
            
        # достигнуто минимальное количество образцов для узла дерева    
        if x.shape[0] <= self.min_samples_split:
            is_it_leaf = True
            
        # узел "чистый", т.е. содержит один класс (тогда его не имеет смысла дальше делить)
        if np.unique(y).shape[0] == 1:
            is_it_leaf = True
            
        if y.shape[0] < min_samples_split:
            is_it_leaf = True
        
        if is_it_leaf:  # создаём лист
            node = self.__create_node(self.__class__.LEAF_TYPE, y)
            self.tree[node_id] = node
            return
        else:  # иначе создаём узел и разделяем его
            features_ids = self.get_feature_ids(x.shape[1])  # выбираем признаки для разделения

            # находим из всех признаков такой, разделение по которому обеспечивает наименьшую неопределённость
            impurity_list = []
            threshold_list = []
            feature_id_list = []
            
            for feature_id in features_ids:
                impurity, threshold = self.__find_threshold(x[:, feature_id], y)
                if threshold is not None:  # для данного признака x целевой признак ни разу не изменяется
                    impurity_list.append(impurity)
                    threshold_list.append(threshold)
                    feature_id_list.append(feature_id)

            if len(impurity_list) == 0:  # тогда это лист
                self.tree[node_id] = self.__create_node(self.__class__.LEAF_TYPE, y)
                return
                
            min_impurity_feature_id = impurity_list.index(min(impurity_list)) # индекс "наилучшего" признака
            feature_threshold = threshold_list[min_impurity_feature_id]       # порог для разделения
            
            # разделяем поддерево в узле
            x_left, x_right, y_left, y_right = self.__div_samples(x, y, min_impurity_feature_id, feature_threshold)
            
            # создаём промежуточный узел
            node = self.__create_node(self.__class__.NON_LEAF_TYPE, y)
            self.tree[node_id] = node
            
            # если после разделения в одной из половин 0, то вторая половина - это лист
            if x_left.shape[0] == 0:
                self.tree[node_id] = self.__create_node(self.__class__.LEAF_TYPE, y_right)
                return
            if x_right.shape[0] == 0:
                self.tree[node_id] = self.__create_node(self.__class__.LEAF_TYPE, y_left)
                return
            if x_left.shape[0] == 0 and x_right.shape[0] == 0:
                return
            # иначе рекурсивный вызов для левого и правого поддеревьев
            if x_left.shape[0] > 0 and x_right.shape[0] > 0:
                if x_left.shape[0] <= self.min_samples_split:
                    self.tree[2 * node_id + 1] = self.__create_node(self.__class__.LEAF_TYPE, y_left)
                    return
                else:
                    self.__fit_node(x_left, y_left, 2 * node_id + 1, depth + 1)
                    
                if x_right.shape[0] <= self.min_samples_split:
                    self.tree[2 * node_id + 2] = self.__create_node(self.__class__.LEAF_TYPE, y_right)
                    return
                else:
                    self.__fit_node(x_right, y_right, 2 * node_id + 2, depth + 1)
            
                      
    def __create_node(self, node_type, y):
        node = dict()
        node[0] = node_type
        # назначаем листу класс - наиболее часто встречающуюся в y величину
        node[1] = np.argmax(np.bincount(y))
        # назначаем вероятность (?)
        node[2] = 0.0
        return node
            
         
    def fit(self, x, y):
        self.num_class = np.unique(y).size  # количество классов признаков. В нашем случае 2 класса - 0 или 1
        self.__fit_node(x, y, 0, 0) 

        
    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

        
    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    
    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [325]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [326]:
x = df.as_matrix(columns=df.columns[1:])  # столбцы, кроме первого - определяющие признаки
y = df.as_matrix(columns=df.columns[:1])  # первый столбец (SeriousDlqin2yrs) - целевой признак
y = y.reshape(y.shape[0])

In [443]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2, max_depth=None)
clf = DecisionTreeClassifier(min_samples_split=2)

In [444]:
my_clf.fit(x, y)

## Проверка скорости работы

In [445]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)

t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

0.681380987167
0.92297911644


## Проверка качества работы

In [ ]:
gkf = KFold(n_splits=5, shuffle=True)

In [ ]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

In [ ]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

# Применить для задачи Titanic 